In [2]:
import calendar
import gc
import os
import sys

import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import scipy
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from implicit.als import AlternatingLeastSquares
import zipfile

/home/mle-user/mle_projects/mle-pr-final/env_recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.getcwd()

'/home/mle-user/mle_projects/mle-pr-final/notebooks'

In [4]:
os.listdir('/home/mle-user/mle_projects/mle-pr-final/data')

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [5]:
arch_path = '/home/mle-user/mle_projects/mle-pr-final/data'
arch_name ='archive.zip'
dest = '/home/mle-user/mle_projects/mle-pr-final/data'

In [6]:
data_path = '/home/mle-user/mle_projects/mle-pr-final/data'

In [7]:
if len(os.listdir(data_path))==1:
     zip_path = os.path.join(arch_path, arch_name)
     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(dest)
else:
     pass

In [8]:
os.listdir(data_path)

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [9]:
category_tree = pd.read_csv(os.path.join(data_path, 'category_tree.csv'))
events = pd.read_csv(os.path.join(data_path, 'events.csv'))
item_properties_part1 = pd.read_csv(os.path.join(data_path, 'item_properties_part1.csv'))
item_properties_part2 = pd.read_csv(os.path.join(data_path, 'item_properties_part2.csv'))

In [10]:
def explore_data(df):
    """
    Исследует структуру данных: типы данных, пропущенные значения, базовые статистики.
    """
    print("===== Основная информация о данных =====")
    print(f"Размер датасета: {df.shape}")
    print("\nТипы данных:")
    print(df.dtypes)
    
    print("\nПропущенные значения:")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])
    
    print("\nБазовая статистика:")
    print(df.describe())
    print(df.head(3))

In [11]:
explore_data(category_tree)

===== Основная информация о данных =====
Размер датасета: (1669, 2)

Типы данных:
categoryid      int64
parentid      float64
dtype: object

Пропущенные значения:
parentid    25
dtype: int64

Базовая статистика:
        categoryid     parentid
count  1669.000000  1644.000000
mean    849.285201   847.571168
std     490.195116   505.058485
min       0.000000     8.000000
25%     427.000000   381.000000
50%     848.000000   866.000000
75%    1273.000000  1291.000000
max    1698.000000  1698.000000
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0


In [12]:
explore_data(events)

===== Основная информация о данных =====
Размер датасета: (2756101, 5)

Типы данных:
timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

Пропущенные значения:
transactionid    2733644
dtype: int64

Базовая статистика:
          timestamp     visitorid        itemid  transactionid
count  2.756101e+06  2.756101e+06  2.756101e+06   22457.000000
mean   1.436424e+12  7.019229e+05  2.349225e+05    8826.497796
std    3.366312e+09  4.056875e+05  1.341954e+05    5098.996290
min    1.430622e+12  0.000000e+00  3.000000e+00       0.000000
25%    1.433478e+12  3.505660e+05  1.181200e+05    4411.000000
50%    1.436453e+12  7.020600e+05  2.360670e+05    8813.000000
75%    1.439225e+12  1.053437e+06  3.507150e+05   13224.000000
max    1.442545e+12  1.407579e+06  4.668670e+05   17671.000000
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214

In [13]:
explore_data(item_properties_part1)

===== Основная информация о данных =====
Размер датасета: (10999999, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  1.100000e+07  1.100000e+07
mean   1.435158e+12  2.333851e+05
std    3.327653e+09  1.348258e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165150e+05
50%    1.433646e+12  2.334990e+05
75%    1.437880e+12  3.501860e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566


In [14]:
explore_data(item_properties_part2)

===== Основная информация о данных =====
Размер датасета: (9275903, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  9.275903e+06  9.275903e+06
mean   1.435156e+12  2.333968e+05
std    3.327970e+09  1.348682e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165175e+05
50%    1.433646e+12  2.334620e+05
75%    1.437880e+12  3.504470e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid property            value
0  1433041200000  183478      561           769062
1  1439694000000  132256      976  n26.400 1135780
2  1435460400000  420307      921  1149317 1257525


In [15]:
category_tree.categoryid.nunique(),category_tree.parentid.nunique(), category_tree.shape[0]

(1669, 362, 1669)

In [16]:
1669/362

4.610497237569061

### заметка

Количество категорий больше в 4,6 раза чем parentid, значит parentid это что то вроде группы, например как для треков был жанр.
Есть 26 категорий без parentid, присвоим им parentid равный 0, так как такого parentid в первоначальных данных нет.

In [17]:
events.head(3)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN


In [18]:
events.event.value_counts()

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [19]:
events.event.value_counts()/events.shape[0]

event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: count, dtype: float64

In [20]:
event_gr = events.groupby('event', as_index=False).agg({'visitorid': 'nunique', 'itemid': 'nunique'})

In [21]:
event_gr

,event,visitorid,itemid
0,addtocart,37722,23903
1,transaction,11719,12025
2,view,1404179,234838


In [22]:
event_gr['visitor_share'] = event_gr['visitorid'] / events['visitorid'].nunique()
event_gr['item_share'] = event_gr['itemid'] / events['itemid'].nunique()

In [23]:
events[~events.transactionid.isna()].head(3)

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0


In [24]:
events[events.transactionid.isna()].shape[0], events.shape[0]

(2733644, 2756101)

In [25]:
item_properties_part1.head(3)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566


In [26]:
item_properties_part1[item_properties_part1['itemid'] == 460429]

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
2122725,1439694000000,460429,202,692224
2595719,1432436400000,460429,839,963847
6332062,1431226800000,460429,917,692224
6490575,1431226800000,460429,364,1058790
8129781,1431226800000,460429,available,0
8368211,1431226800000,460429,283,1103756 9705 963847
8566995,1433646000000,460429,283,1103756 9705 963847
8752572,1431226800000,460429,6,9705
8753668,1431226800000,460429,776,674847


In [27]:
item_properties_part2[item_properties_part2['itemid'] == 460429]

,timestamp,itemid,property,value
151187,1438484400000,460429,283,9705 963847 692224
349970,1442113200000,460429,283,9705 963847 692224
548753,1439694000000,460429,283,9705 963847 692224
563950,1431226800000,460429,888,692224
747536,1436065200000,460429,283,9705 963847 692224
788083,1435460400000,460429,764,1285872
1266667,1435460400000,460429,227,1103756 9705
1465450,1431226800000,460429,227,1103756 9705
1664233,1431831600000,460429,227,1103756 9705
1863016,1433041200000,460429,227,1103756 9705


In [28]:
item_properties_part1.property.value_counts(ascending=False)[0:10]

property
888           1629817
790            970800
available      817387
categoryid     426305
6              343207
283            323681
776            311654
678            261829
364            256340
202            242984
Name: count, dtype: int64

In [29]:
item_properties_part2.head(3)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525


In [30]:
category_tree[category_tree['categoryid'] == 1338]

,categoryid,parentid
742,1338,1278.0


In [31]:
category_tree[category_tree['parentid'] == 1278.0]

,categoryid,parentid
170,578,1278.0
742,1338,1278.0
1134,1374,1278.0


In [32]:
category_tree.head(3)

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0


In [33]:
categoryid_prop1 = item_properties_part1[item_properties_part1['property'] == 'categoryid']
categoryid_prop2 = item_properties_part2[item_properties_part2['property'] == 'categoryid']
is_available1 = item_properties_part1[item_properties_part1['property'] == 'available']
is_available2 = item_properties_part1[item_properties_part1['property'] == 'available']

In [34]:
is_available = pd.concat([is_available1, is_available2])
category_prop = pd.concat([categoryid_prop1, categoryid_prop2])

In [35]:
is_available['value'] = is_available['value'].astype(int)

In [36]:
import gc
del item_properties_part1
del item_properties_part2
gc.collect()

0

In [37]:
# big_tab = ps.sqldf(""" select t1.*,
#                    t2.timestamp as is_avail_ts, 
#                    t2.value as is_available,
#                    t4.parentid as gr
#                    from events t1
#                    left join is_available t2
#                    on t1.itemid = t2.itemid
#                    left join category_prop t3
#                    on t1.itemid = t3.itemid 
#                    left join category_tree t4
#                    on t4.categoryid = t3.value""")

In [45]:
merged = events.merge(
    is_available,
    how='left',
    left_on='itemid',
    right_on='itemid',
    suffixes=('', '_t2')
)

# Переименуем столбцы из is_available
merged.rename(columns={'timestamp': 'is_avail_ts', 'value': 'is_available'}, inplace=True)

# Второй join: с category_prop (t3)
merged = merged.merge(
    category_prop,
    how='left',
    left_on='itemid',
    right_on='itemid',
    suffixes=('', '_t3')
)


In [52]:
category_tree['categoryid'] = category_tree['categoryid'].astype(str)

In [53]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58179328 entries, 0 to 58179327
Data columns (total 11 columns):
 #   Column         Dtype  
---  ------         -----  
 0   is_avail_ts    int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
 5   timestamp_t2   float64
 6   property       object 
 7   is_available   float64
 8   timestamp      float64
 9   property_t3    object 
 10  value          object 
dtypes: float64(4), int64(3), object(4)
memory usage: 4.8+ GB


In [57]:
merged = merged.merge(
    category_tree,
    how='left',
    left_on='value',  # предполагается, что 'value' из category_prop используется для связи
    right_on='categoryid',
    suffixes=('', '_t4')
)

: 

In [56]:
merged

,is_avail_ts,visitorid,event,itemid,transactionid,timestamp_t2,property,is_available,timestamp,property_t3,value
0,1433221332117,257597,view,355908,NaN,1.431832e+12,available,1.0,1.431227e+12,categoryid,1173
1,1433221332117,257597,view,355908,NaN,1.440904e+12,available,1.0,1.431227e+12,categoryid,1173
2,1433221332117,257597,view,355908,NaN,1.436065e+12,available,1.0,1.431227e+12,categoryid,1173
3,1433221332117,257597,view,355908,NaN,1.437880e+12,available,1.0,1.431227e+12,categoryid,1173
4,1433221332117,257597,view,355908,NaN,1.431832e+12,available,1.0,1.431227e+12,categoryid,1173
...,...,...,...,...,...,...,...,...,...,...,...
58179323,1438397820527,1251746,view,78144,NaN,1.436065e+12,available,1.0,1.431227e+12,categoryid,969
58179324,1438397820527,1251746,view,78144,NaN,1.440904e+12,available,0.0,1.431227e+12,categoryid,969
58179325,1438398530703,1184451,view,283392,NaN,NaN,NaN,NaN,1.433041e+12,categoryid,1589
58179326,1438400163914,199536,view,152913,NaN,1.431227e+12,available,0.0,1.431227e+12,categoryid,469


In [41]:
events_av = events.merge(
    is_available,
    how='left',
    left_on='itemid',
    right_on='itemid',
    suffixes=('', '_is_ava')
)

In [42]:
events_av.rename(columns={'value': 'is_available'}, inplace=True).drop()

,timestamp,visitorid,event,itemid,transactionid,timestamp_is_ava,property,value
0,1433221332117,257597,view,355908,NaN,1.431832e+12,available,1.0
1,1433221332117,257597,view,355908,NaN,1.440904e+12,available,1.0
2,1433221332117,257597,view,355908,NaN,1.436065e+12,available,1.0
3,1433221332117,257597,view,355908,NaN,1.437880e+12,available,1.0
4,1433221332117,257597,view,355908,NaN,1.431832e+12,available,1.0
...,...,...,...,...,...,...,...,...
25286102,1438397820527,1251746,view,78144,NaN,1.436065e+12,available,1.0
25286103,1438397820527,1251746,view,78144,NaN,1.440904e+12,available,0.0
25286104,1438398530703,1184451,view,283392,NaN,NaN,NaN,NaN
25286105,1438400163914,199536,view,152913,NaN,1.431227e+12,available,0.0


In [39]:
events['value'] = events['value'].fillna(2).astype(int)

KeyError: 'value'

In [ ]:
# Первый join: events (t1) с is_available (t2)
merged = events.merge(
    is_available,
    how='left',
    left_on='itemid',
    right_on='itemid',
    suffixes=('', '_t2')
)

# Переименуем столбцы из is_available
merged.rename(columns={'timestamp': 'is_avail_ts', 'value': 'is_available'}, inplace=True)

# Второй join: с category_prop (t3)
merged = merged.merge(
    category_prop,
    how='left',
    left_on='itemid',
    right_on='itemid',
    suffixes=('', '_t3')
)

# Третий join: с category_tree (t4)
merged = merged.merge(
    category_tree,
    how='left',
    left_on='value_t3',  # предполагается, что 'value' из category_prop используется для связи
    right_on='categoryid',
    suffixes=('', '_t4')
)

# Выбираем нужные столбцы и переименовываем 'parentid' в 'gr'
big_tab = merged[
    [col for col in events.columns] +  # все столбцы из events
    ['is_avail_ts', 'is_available', 'parentid']
]
big_tab.rename(columns={'parentid': 'gr'}, inplace=True)

# Результат сохраняется в big_tab